In [9]:
import torch
from torch import nn
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
import torch
import torch.nn as nn

class GRU(nn.Module):
    def __init__(self, in_dim ,hidden, depth = 2):
        super(GRU, self).__init__()
        self.hidden = hidden
        self.depth = depth
        self.gru = nn.GRU(in_dim, self.hidden, num_layers = self.depth, batch_first=True)
        
    def forward(self, x):
        out, self.hidden_state = self.gru(x, self.hidden_state)
        return out
    
    def init_hidden(self, batch_size):
        self.hidden_state = torch.zeros(self.depth, batch_size, self.hidden)

In [8]:
test = nn.GRU(1, 16, num_layers = 1)

In [31]:
print(test.weight_hh_l0.data.shape, test.weight_ih_l0.data.shape)

torch.Size([48, 16]) torch.Size([48, 1])
